In [1]:
import jax
jax.__version__

'0.4.14'

## DeviceArray properties

In [2]:
import numpy as np
import jax.numpy as jnp

In [3]:
np.array([1, 42, 31337])

array([    1,    42, 31337])

In [4]:
jnp.array([1, 42, 31337])

Array([    1,    42, 31337], dtype=int32)

In [5]:
np.sum([1, 42, 31337])

31380

In [6]:
try:
  jnp.sum([1, 42, 31337])
except TypeError as e:
  print(e)

sum requires ndarray or scalar arguments, got <class 'list'> at position 0.


In [7]:
jnp.sum(jnp.array([1, 42, 31337]))

Array(31380, dtype=int32)

In [8]:
arr = jnp.array([1, 42, 31337])

In [9]:
arr.ndim

1

In [10]:
arr.shape

(3,)

In [11]:
arr.dtype

dtype('int32')

In [12]:
arr.size

3

In [13]:
arr.nbytes

12

## Devices

In [14]:
import jax

In [15]:
jax.devices()

[gpu(id=0)]

In [16]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [17]:
jax.device_count('gpu')

1

In [18]:
jax.local_devices()

[gpu(id=0)]

In [19]:
arr = jnp.array([1, 42, 31337])

In [20]:
arr.device()

gpu(id=0)

In [21]:
arr_cpu = jax.device_put(arr, jax.devices('cpu')[0])

In [22]:
arr_cpu.device()

CpuDevice(id=0)

In [23]:
arr.device()

gpu(id=0)

In [24]:
arr_host = jax.device_get(arr)

In [25]:
type(arr_host)

numpy.ndarray

In [26]:
arr_host

array([    1,    42, 31337], dtype=int32)

In [27]:
arr + arr_cpu

Array([    2,    84, 62674], dtype=int32)

In [28]:
arr_gpu = jax.device_put(arr, jax.devices('gpu')[0])

In [29]:
arr_gpu.device()

gpu(id=0)

In [30]:
try:
  arr_gpu + arr_cpu
except ValueError as e:
  print(e)

Received incompatible devices for jitted computation. Got argument x1 of jax.numpy.add with shape int32[3] and device ids [0] on platform GPU and argument x2 of jax.numpy.add with shape int32[3] and device ids [0] on platform CPU


## Asyncronous dispatch

In [31]:
import jax

In [32]:
a = jnp.array(range(1000000)).reshape((1000,1000))

In [33]:
a.shape

(1000, 1000)

In [34]:
a.device()

gpu(id=0)

In [35]:
%time x = jnp.dot(a,a)

CPU times: user 39.9 ms, sys: 1.04 ms, total: 41 ms
Wall time: 85.1 ms


In [36]:
%time x = jnp.dot(a,a).block_until_ready()

CPU times: user 1.19 ms, sys: 45 µs, total: 1.24 ms
Wall time: 6.45 ms


In [37]:
%time x = np.asarray(jnp.dot(a,a))

CPU times: user 8.27 ms, sys: 168 µs, total: 8.44 ms
Wall time: 7.62 ms


In [38]:
a_cpu = jax.device_put(a, jax.devices('cpu')[0])

In [39]:
a_cpu.device()

CpuDevice(id=0)

In [40]:
%time x = jnp.dot(a_cpu,a_cpu).block_until_ready()

CPU times: user 333 ms, sys: 527 µs, total: 334 ms
Wall time: 192 ms


## GPU diagnostics

In [41]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [42]:
#!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [43]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Oct  2 13:02:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W /  70W |  11743MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Immutability

In [44]:
import numpy as np
import jax.numpy as jnp

In [45]:
a_jnp = jnp.array(range(10))
a_np  = np.array(range(10))

In [46]:
a_jnp

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [47]:
a_np[5], a_jnp[5]

(5, Array(5, dtype=int32))

In [48]:
a_np[5] = 100

In [49]:
a_np[5]

100

In [50]:
try:
  a_jnp[5] = 100
except TypeError as e:
  print(e)

'<class 'jaxlib.xla_extension.ArrayImpl'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html


In [51]:
a_jnp = a_jnp.at[5].set(100)

In [52]:
a_jnp[5]

Array(100, dtype=int32)

In [53]:
a_jnp[42]

Array(9, dtype=int32)

In [54]:
a_jnp.at[42].get()

Array(9, dtype=int32)

In [55]:
a_jnp.at[42].get(mode='clip')

Array(9, dtype=int32)

In [56]:
a_jnp.at[42].get(mode='drop')

Array(-2147483648, dtype=int32)

In [57]:
a_jnp.at[42].get(mode='fill', fill_value=-1)

Array(-1, dtype=int32)

In [58]:
a_jnp = a_jnp.at[42].set(100)
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8,   9], dtype=int32)

In [59]:
a_jnp = a_jnp.at[42].set(100, mode='clip')
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8, 100], dtype=int32)

## Working with float64

In [60]:
# this only works on startup!
from jax.config import config
config.update("jax_enable_x64", True)

In [61]:
import jax
import jax.numpy as jnp

In [62]:
# this will not work on TPU backend. Try using CPU or GPU.
x = jnp.array(range(10), dtype=jnp.float64)
x.dtype

dtype('float64')

In [63]:
x.device()

gpu(id=0)

In [64]:
xc = jax.device_put(x, jax.devices('cpu')[0])

In [65]:
xc.device()

CpuDevice(id=0)

In [66]:
xc.dtype

dtype('float64')

In [67]:
xb16 = jnp.array(range(10), dtype=jnp.bfloat16)
xb16.dtype

dtype(bfloat16)

In [68]:
xb16.nbytes

20

In [69]:
x16 = jnp.array(range(10), dtype=jnp.float16)
x16.dtype

dtype('float16')

In [70]:
x16.nbytes

20

In [71]:
xb16+x16

Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)

In [72]:
xb16+xb16

Array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18], dtype=bfloat16)

## jax.numpy & jax.lax

In [84]:
config.update("jax_enable_x64", False)

In [85]:
import jax.numpy as jnp
from jax import lax
from jax import random

In [86]:
jnp.add(42, 42.0)

Array(84., dtype=float32, weak_type=True)

In [87]:
jnp.add(42.0, 42.0)

Array(84., dtype=float32, weak_type=True)

In [88]:
try:
   lax.add(42, 42.0)
except TypeError as e:
  print(e)

lax.add requires arguments to have the same dtypes, got int32, float32. (Tip: jnp.add is a similar function that does automatic type promotion on inputs).


In [89]:
lax.add(jnp.float32(42), 42.0)

Array(84., dtype=float32)

In [90]:
def random_augmentation(image, augmentations, rng_key):
   '''A function that applies a random transformation to an image'''
   augmentation_index = random.randint(key=rng_key, minval=0, maxval=len(augmentations), shape=())
   augmented_image = lax.switch(augmentation_index, augmentations, image)
   return augmented_image

In [91]:
add_noise_func = lambda x: x+10
horizontal_flip_func = lambda x: x+1
rotate_func = lambda x: x+2
adjust_colors_func = lambda x: x+3

augmentations = [
   add_noise_func,
   horizontal_flip_func,
   rotate_func,
   adjust_colors_func
]


In [92]:
image = jnp.array(range(100))

In [93]:
random_augmentation(image, augmentations, random.PRNGKey(211))

Array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)